vtensorflow core implementation** of [mriberodiaz/federatedRecsys](https://github.com/mriberodiaz/federatedRecsys/blob/master/utilsMovie.py)

In [1]:
from src.data_examples.ex2_data_loader import ExampleDataLoader
from src.tf_utils import df_to_tfds, df_ls_to_tfds_ls
from dataclasses import dataclass

import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
RAND_SEED = 1337
label_col_name = 'title'

In [3]:
data_loader = ExampleDataLoader()
data_loader.download().load().clean()

using cached file cache\datasets\movielens\ml-100k.zip
extracting zip file content:
file already exists: data\ex2\ml-100k
file already exists: data\ex2\ml-100k\allbut.pl
file already exists: data\ex2\ml-100k\mku.sh
file already exists: data\ex2\ml-100k\README
file already exists: data\ex2\ml-100k\u.data
file already exists: data\ex2\ml-100k\u.genre
file already exists: data\ex2\ml-100k\u.info
file already exists: data\ex2\ml-100k\u.item
file already exists: data\ex2\ml-100k\u.occupation
file already exists: data\ex2\ml-100k\u.user
file already exists: data\ex2\ml-100k\u1.base
file already exists: data\ex2\ml-100k\u1.test
file already exists: data\ex2\ml-100k\u2.base
file already exists: data\ex2\ml-100k\u2.test
file already exists: data\ex2\ml-100k\u3.base
file already exists: data\ex2\ml-100k\u3.test
file already exists: data\ex2\ml-100k\u4.base
file already exists: data\ex2\ml-100k\u4.test
file already exists: data\ex2\ml-100k\u5.base
file already exists: data\ex2\ml-100k\u5.test
fil

c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column release_date - only the converter will be used.

In [4]:
def split_dataframe(df, frac=.8, seed=None):
  df_train = df.sample(frac=frac, random_state=seed)
  df_test = df.drop(df_train.index)
  return df_train, df_test

In [5]:
df_train, df_test = split_dataframe(data_loader.df, seed=RAND_SEED)
tfds_train, tfds_test = df_ls_to_tfds_ls([df_train, df_test], label_col_name)

df_train

c:\Users\62896\Documents\GitHub\FederatedLearning\src\tf_utils.py:12: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  c_df = {key: value[:,tf.newaxis] for key, value in df.items()}
c:\Users\62896\Documents\GitHub\FederatedLearning\src\tf_utils.py:12: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  c_df = {key: value[:,tf.newaxis] for key, value in df.items()}


,user_id,age,sex,occupation,zip_code,rating,movie_id,title
36649,585,20,M,student,79508,3.0,154,Dirty Dancing (1987)
2478,300,24,M,student,55439,3.0,691,"American President, The (1995)"
82838,394,43,M,other,44092,3.0,891,Flubber (1997)
69729,877,50,F,educator,98027,3.0,922,Raise the Red Lantern (1991)
36560,558,69,M,executive,10022,4.0,651,Rosencrantz and Guildenstern Are Dead (1990)
...,...,...,...,...,...,...,...,...
35200,553,32,M,scientist,62901,2.0,67,"Crow, The (1994)"
311,285,27,M,student,15217,5.0,380,Muriel's Wedding (1994)
52771,858,18,F,other,06492,4.0,275,Leaving Las Vegas (1995)
29789,6,57,M,administrator,91344,2.0,200,Evil Dead II (1987)


In [6]:
from src.tf_layer_constructors import (
  gen_multihot_categorical_encoding_layer, 
  gen_normalization_layer, 
  gen_string_lookup, 
  gen_string_embeddings
)
from src.custom_types import TypeEnum
from src.utils import dict_to_flat_list

In [7]:
exclude = ['user_id', 'title']

data_features = [i for j in dict_to_flat_list(data_loader.data_feature_cols, ls=[]) for i in j]
data_features = [i for i in data_features if i not in exclude]

data_features 

['age', 'sex', 'occupation', 'zip_code', 'rating', 'movie_id']

In [8]:
input_interface_layers = {}

for feature in data_features:
  input_interface_layers[feature] = tf.keras.layers.Input(
    shape=(1,), name=f'input_{feature}', dtype=data_loader.fetch_datatype(feature)
  )

input_interface_layers

TypeError: ExampleDataLoader.fetch_datatype() takes 1 positional argument but 2 were given

In [ ]:
# categorical_col_names = ['sex', 'occupation', 'zip_code']
# integer_col_names = ['age']
# float_col_names = ['rating']


# def generate_layers(tfds, label_col_name):
#   inputs = {}
#   encoded_layers = {}

#   title_indexer = None

#   for col_name in categorical_col_names:
#     input_categorical = tf.keras.Input(shape=(1,), name=col_name, dtype=fetch_datatype_from_loader(data_loader, col_name))
#     inputs[col_name] = input_categorical
#     encoder, indexer = gen_string_embeddings(tfds, col_name=col_name, vocabulary=data_loader.get_feature_values(feature_col=col_name))
#     encoded_layer = encoder(input_categorical)
#     encoded_layers[col_name] = encoded_layer

#   for col_name in integer_col_names:
#     input_integer = tf.keras.Input(shape=(1,), name=col_name, dtype=fetch_datatype_from_loader(data_loader, col_name))
#     inputs[col_name] = input_integer
#     encoder = gen_normalization_layer(tfds, col_name=col_name)
#     encoded_layer = encoder(input_integer)
#     encoded_layers[col_name] = encoded_layer

#   for col_name in float_col_names:
#     input_float = tf.keras.Input(shape=(1,), name=col_name, dtype=fetch_datatype_from_loader(data_loader, col_name))
#     inputs[col_name] = input_float
#     encoder = gen_normalization_layer(tfds, col_name=col_name)
#     encoded_layer = encoder(input_float)
#     encoded_layers[col_name] = encoded_layer

#   for col_name in ['title']:
#     input_string = tf.keras.Input(shape=(1,), name=col_name, dtype=fetch_datatype_from_loader(data_loader, col_name))
#     inputs[col_name] = input_string
#     encoder, indexer = gen_string_embeddings(tfds, col_name=col_name)
#     encoded_layer = encoder(input_string)
#     encoded_layers[col_name] = encoded_layer
#     title_indexer = indexer

#   return inputs, encoded_layers


# input_layers, encoded_layers = generate_layers(tfds_train, 'title')
# input_layers, encoded_layers